In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 경로 설정

In [ ]:
import os

In [ ]:
# 디렉토리 변경
os.chdir('drive/My Drive/Colab Notebooks/DACON/AIFrenz_Season2/')

# 데이터 불러오기

In [ ]:
import numpy as np

In [ ]:
data = np.load('./Data/dacon_npy/train.npy')

In [ ]:
x = data[:, :, :, :-1]
y = data[:, :, :, -1].reshape(-1, 40, 40, 1)

In [ ]:
del data

# 훈련 데이터와 검증 데이터로 분리

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.025, random_state=7777)
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((74436, 40, 40, 14),
 (74436, 40, 40, 1),
 (1909, 40, 40, 14),
 (1909, 40, 40, 1))

In [ ]:
del x, y

# 평가 함수 정의
- 규칙에 정의된 함수

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def mae_over_fscore(y_true, y_pred):
    '''
    y_true: sample_submission.csv 형태의 실제 값
    y_pred: sample_submission.csv 형태의 예측 값
    '''

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    # 실제값이 0.1 이상인 픽셀의 위치 확인
    IsGreaterThanEqualTo_PointOne = y_true >= 0.1
    
    # 실제 값에 결측값이 없는 픽셀의 위치 확인 
    IsNotMissing = y_true >= 0
    
    # mae 계산
    mae = np.mean(np.abs(y_true[IsGreaterThanEqualTo_PointOne] - y_pred[IsGreaterThanEqualTo_PointOne]))
    
    # f1_score 계산 위해, 실제값에 결측값이 없는 픽셀에 대해 1과 0으로 값 변환
    y_true = np.where(y_true[IsNotMissing] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[IsNotMissing] >= 0.1, 1, 0)
    
    # f1_score 계산    
    f_score = f1_score(y_true, y_pred) 
    # f1_score가 0일 나올 경우를 대비하여 소량의 값 (1e-07) 추가 
    return mae / (f_score + 1e-07) 

# 손실 함수 정의
- 베이스라인에 정의된 함수

In [ ]:
import tensorflow as tf

In [ ]:
def mae(y_true, y_pred):    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def score(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

# 모델 설계 및 학습
- 강수량을 예측할 지점에서 멀리 있는 데이터일 수록 영향이 적어져 풀링레이어가 결과 예측에 방해가 될 것으로 판단

- convolution과 residual block만으로 모델 설계

In [ ]:
def create_model():
    inputs = tf.keras.layers.Input(x_train.shape[1:])
    
    bn = tf.keras.layers.BatchNormalization()(inputs)
    conv0 = tf.keras.layers.Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn = tf.keras.layers.BatchNormalization()(conv0)
    conv = tf.keras.layers.Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat = tf.keras.layers.concatenate([conv0, conv], axis=3)
    
    bn = tf.keras.layers.BatchNormalization()(concat)
    conv = tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat = tf.keras.layers.concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn = tf.keras.layers.BatchNormalization()(concat)
        conv = tf.keras.layers.Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat = tf.keras.layers.concatenate([concat, conv], axis=3)
    
    bn = tf.keras.layers.BatchNormalization()(concat)
    outputs = tf.keras.layers.Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
model = create_model()
model.compile(loss='mae', optimizer='adam', metrics=[score, fscore_keras])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 40, 14)] 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 40, 40, 14)   56          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 40, 40, 256)  3840        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 40, 40, 256)  1024        conv2d[0][0]                     
______________________________________________________________________________________________

# 콜백 함수 정의

In [ ]:
callbacks_list = [
                  tf.keras.callbacks.EarlyStopping(
                      monitor='val_maeOverFscore_keras',
                      patience=3,
                      restore_best_weights=True
                      ),
                  tf.keras.callbacks.ReduceLROnPlateau(
                      monitor='val_maeOverFscore_keras',
                      patience=3,
                      factor=0.8
                      )
                  #tf.keras.callbacks.ModelCheckpoint(
                  #    filepath='models/model' + str(model_number)+'.h5',
                  #    monitor='val_maeOverFscore_keras',
                  #    save_best_only=True
                  #    )
                  ]

# 모델 훈련

In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_valid, y_valid), callbacks=callbacks_list)

Epoch 1/50
582/582 [==============================] - 210s 360ms/step - loss: 44.0946 - score: 2.6872 - fscore_keras: 0.6372 - val_loss: 65.8430 - val_score: 2.2357 - val_fscore_keras: 0.6397 - lr: 0.0010
Epoch 2/50
582/582 [==============================] - 207s 356ms/step - loss: 44.0700 - score: 2.0427 - fscore_keras: 0.6985 - val_loss: 65.8357 - val_score: 1.8135 - val_fscore_keras: 0.7189 - lr: 0.0010
Epoch 3/50
582/582 [==============================] - 209s 359ms/step - loss: 44.0670 - score: 1.9326 - fscore_keras: 0.7176 - val_loss: 65.8371 - val_score: 1.8556 - val_fscore_keras: 0.7103 - lr: 0.0010
Epoch 4/50
582/582 [==============================] - 208s 357ms/step - loss: 44.0661 - score: 1.9015 - fscore_keras: 0.7221 - val_loss: 65.8363 - val_score: 1.9276 - val_fscore_keras: 0.6801 - lr: 0.0010
Epoch 5/50
582/582 [==============================] - 208s 357ms/step - loss: 44.0652 - score: 1.8741 - fscore_keras: 0.7247 - val_loss: 65.8322 - val_score: 1.7112 - val_fscore_ke

In [ ]:
del x_train, y_train

# 모델 저장

In [ ]:
model.save('./Model/ResNet/ResNet without KFold.h5')

# 가채점

In [ ]:
x_test = np.load('./Data/dacon_npy/test.npy')

In [ ]:
pred = model.predict(x_test)

In [ ]:
del x_test

# Submission

In [ ]:
import pandas as pd

In [ ]:
submission = pd.read_csv('./Submission/sample_submission.csv')

In [ ]:
submission.iloc[:,1:] = pred.reshape(-1,1600)
submission

,id,px_1,px_2,px_3,px_4,px_5,px_6,px_7,px_8,px_9,px_10,px_11,px_12,px_13,px_14,px_15,px_16,px_17,px_18,px_19,px_20,px_21,px_22,px_23,px_24,px_25,px_26,px_27,px_28,px_29,px_30,px_31,px_32,px_33,px_34,px_35,px_36,px_37,px_38,px_39,...,px_1561,px_1562,px_1563,px_1564,px_1565,px_1566,px_1567,px_1568,px_1569,px_1570,px_1571,px_1572,px_1573,px_1574,px_1575,px_1576,px_1577,px_1578,px_1579,px_1580,px_1581,px_1582,px_1583,px_1584,px_1585,px_1586,px_1587,px_1588,px_1589,px_1590,px_1591,px_1592,px_1593,px_1594,px_1595,px_1596,px_1597,px_1598,px_1599,px_1600
0,029858_01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.9862,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,029858_02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.05488,0.440251,0.000000,0.000000,0.240220,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.387755,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.418119,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,029858_03,0.000000,0.000000,0.000000,0.409637,0.379366,0.000000,0.0,0.0,0.0,0.017526,0.0,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.131083,0.846188,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,029858_05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.643427,1.655594,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.499123,2.003746,5.166891,2.276213,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,029858_07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,1.016005,1.496318,1.546383,2.311344,3.555382,4.098023,2.131316,0.691932,0.492428,1.255971,2.602510,5.091553,4.985069,3.78241,5.565695,8.338867,11.844794,19.363216,21.732315,22.285822,11.20038,6.593974,12.580007,19.050917,7.476026,1.269255,0.100233,0.075599,0.26082,0.646883,1.538488,2.413249,1.683983,0.717243,0.725052,1.880427,5.503625,4.805448,3.08319,1.638869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
del pred

In [ ]:
submission.to_csv('./Submission/ResNet without KFold.csv', index=False)